In [14]:
import sys
import os
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import yaml
from datetime import datetime as dt, date
import requests
import logging
from datetime import datetime, timedelta
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut
from src.databaseService.Parser import Parser_AV
from src.databaseService.CleanData import CleanData
from src.common.AssetDataService import AssetDataService

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]
# Usage
logger.info("This will print to the notebook’s output cell")

This will print to the notebook’s output cell


In [15]:
# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [16]:
ticker = "GILD"
ts = TimeSeries(key=apiKey, output_format='pandas')
fd = FundamentalData(key=apiKey, output_format='pandas')

datashareprice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
loadad_overview, _ = fd.get_company_overview(symbol=ticker)

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+ticker+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+ticker+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+apiKey
earningsData = requests.get(url).json()

if incStatementData=={} or cashFlowData == {} or balanceSheetData == {} or earningsData == {}:
    raise ImportError(f"Empty Financial Data")

In [17]:
res = Parser_AV(sharepriceData=datashareprice).parse_shareprice()

res.tail(10)

,Date,Open,High,Low,Close,AdjClose,Volume,Dividends,Splits
6410,2025-04-28,103.855,106.540,103.135,106.34,106.34,9458853.0,0.0,1.0
6411,2025-04-29,105.680,107.170,104.110,104.21,104.21,7646381.0,0.0,1.0
6412,2025-04-30,105.330,106.810,103.800,106.54,106.54,9628528.0,0.0,1.0
6413,2025-05-01,105.505,105.860,103.160,103.25,103.25,7270898.0,0.0,1.0
6414,2025-05-02,104.870,105.170,102.475,103.68,103.68,7033837.0,0.0,1.0
6415,2025-05-05,103.720,103.845,102.455,102.81,102.81,5487254.0,0.0,1.0
6416,2025-05-06,102.330,102.730,97.330,97.88,97.88,11409437.0,0.0,1.0
6417,2025-05-07,97.870,99.700,97.330,98.90,98.90,8126794.0,0.0,1.0
6418,2025-05-08,97.250,98.440,95.295,98.16,98.16,12994843.0,0.0,1.0
6419,2025-05-09,98.070,98.650,96.200,96.91,96.91,13141070.0,0.0,1.0


In [18]:
loadad_about, loaded_sector = Parser_AV(overview=loadad_overview).parse_overview()

print("About: ", loadad_about)
print("Sector: ", loaded_sector)

About:  {'Symbol': 'GILD', 'AssetType': 'Common Stock', 'Name': 'Gilead Sciences Inc', 'Description': 'Gilead Sciences, Inc. is an American biopharmaceutical company headquartered in Foster City, California, that focuses on researching and developing antiviral drugs used in the treatment of HIV, hepatitis B, hepatitis C, and influenza, including Harvoni and Sovaldi.', 'CIK': '882095', 'Exchange': 'NASDAQ', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'LIFE SCIENCES', 'Industry': 'BIOLOGICAL PRODUCTS, (NO DISGNOSTIC SUBSTANCES)', 'Address': '333 LAKESIDE DR, FOSTER CITY, CA, US', 'OfficialSite': 'https://www.gilead.com', 'FiscalYearEnd': 'December', 'LatestQuarter': '2025-03-31', 'MarketCapitalization': '122209329000', 'EBITDA': '13586750000', 'PERatio': '20.36', 'PEGRatio': '0.205', 'BookValue': '15.32', 'DividendPerShare': '3.1', 'DividendYield': '0.0326', 'EPS': '4.76', 'RevenuePerShareTTM': '23.05', 'ProfitMargin': '0.208', 'OperatingMarginTTM': '0.385', 'ReturnOnAssetsTTM': '0.12

In [19]:
financials_annually, financials_quarterly = Parser_AV(
    incStatementData = incStatementData, 
    cashFlowData = cashFlowData, 
    balanceSheetData = balanceSheetData, 
    earningsData = earningsData
).parse_financials()

In [20]:
financials_annually.tail(10)

,fiscalDateEnding,reportedEPS,reportedCurrency_x,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_y
19,2015-12-31 00:00:00+00:00,12.63,USD,2.863300e+10,3.263900e+10,4.006000e+09,4.006000e+09,2.219300e+10,3.426000e+09,3.014000e+09,...,1.874000e+09,None,None,None,None,-1.000200e+10,None,2.824000e+09,-67000000.0,1.810600e+10
20,2016-12-31 00:00:00+00:00,11.56,USD,2.612900e+10,3.039000e+10,4.261000e+09,4.261000e+09,1.763300e+10,3.398000e+09,5.098000e+09,...,2.455000e+09,None,None,None,None,-1.100100e+10,None,-4.622000e+09,41000000.0,1.348800e+10
21,2017-12-31 00:00:00+00:00,8.84,USD,2.173600e+10,2.610700e+10,4.371000e+09,4.371000e+09,1.412400e+10,3.878000e+09,3.734000e+09,...,2.731000e+09,None,None,None,None,-9.540000e+08,None,-6.410000e+08,137000000.0,4.644000e+09
22,2018-12-31 00:00:00+00:00,6.67,USD,1.727400e+10,2.212700e+10,4.853000e+09,4.853000e+09,8.200000e+09,4.056000e+09,5.018000e+09,...,2.971000e+09,None,None,None,None,-2.900000e+09,None,1.035200e+10,-85000000.0,5.460000e+09
23,2019-12-31 00:00:00+00:00,6.63,USD,1.777400e+10,2.244900e+10,4.675000e+09,4.675000e+09,4.287000e+09,3.597000e+09,9.106000e+09,...,3.222000e+09,None,None,None,None,-1.749000e+09,None,-6.307000e+09,NaN,5.364000e+09
24,2020-12-31 00:00:00+00:00,7.09,USD,2.011700e+10,2.468900e+10,4.572000e+09,4.572000e+09,7.029000e+09,4.356000e+09,5.039000e+09,...,3.449000e+09,None,None,None,None,-1.583000e+09,None,-5.677000e+09,NaN,8.900000e+07
25,2021-12-31 00:00:00+00:00,7.29,USD,2.070400e+10,2.730500e+10,6.601000e+09,6.601000e+09,7.406000e+09,4.511000e+09,5.363000e+09,...,3.605000e+09,None,None,None,None,-5.460000e+08,None,-6.240000e+08,NaN,6.201000e+09
26,2022-12-31 00:00:00+00:00,7.27,USD,2.162400e+10,2.728100e+10,5.657000e+09,5.657000e+09,7.330000e+09,4.895000e+09,4.977000e+09,...,3.709000e+09,None,None,None,None,-1.396000e+09,None,1.370000e+08,NaN,4.566000e+09
27,2023-12-31 00:00:00+00:00,7.04,USD,2.711600e+10,2.711600e+10,5.804000e+09,5.804000e+09,7.605000e+09,6.502000e+09,5.718000e+09,...,3.809000e+09,None,None,None,None,-1.000000e+09,None,NaN,NaN,5.613000e+09
28,2024-12-31 00:00:00+00:00,4.61,USD,2.250300e+10,2.875400e+10,6.251000e+09,6.251000e+09,1.662000e+09,6.091000e+09,5.907000e+09,...,3.918000e+09,None,None,None,None,-1.150000e+09,None,NaN,NaN,4.800000e+08


In [21]:
financials_quarterly.tail(10)

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,reportedCurrency_x,grossProfit,totalRevenue,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_y
107,2022-12-31 00:00:00+00:00,2023-02-02 00:00:00+00:00,1.67,1.5000,0.1700,11.3333,post-market,USD,5.993000e+09,7.389000e+09,...,9.150000e+08,None,None,None,None,-792000000.0,None,638000000.0,NaN,1.633000e+09
108,2023-03-31 00:00:00+00:00,2023-04-27 00:00:00+00:00,1.37,1.5400,-0.1700,-11.0390,post-market,USD,4.951000e+09,6.352000e+09,...,9.690000e+08,None,None,None,None,-400000000.0,None,-488000000.0,NaN,9.850000e+08
109,2023-06-30 00:00:00+00:00,2023-08-03 00:00:00+00:00,1.66,1.6400,0.0200,1.2195,post-market,USD,5.157000e+09,6.599000e+09,...,9.440000e+08,None,None,None,None,-150000000.0,None,754000000.0,NaN,1.039000e+09
110,2023-09-30 00:00:00+00:00,2023-11-07 00:00:00+00:00,2.29,1.9200,0.3700,19.2708,post-market,USD,5.485000e+09,7.050000e+09,...,9.530000e+08,None,None,None,None,-300000000.0,None,NaN,NaN,2.172000e+09
111,2023-12-31 00:00:00+00:00,2024-02-06 00:00:00+00:00,1.72,1.7600,-0.0400,-2.2727,post-market,USD,5.555000e+09,7.115000e+09,...,9.430000e+08,None,None,None,None,-150000000.0,None,NaN,NaN,1.417000e+09
112,2024-03-31 00:00:00+00:00,2024-04-25 00:00:00+00:00,-1.32,-1.4900,0.1700,11.4094,post-market,USD,5.134000e+09,6.686000e+09,...,9.900000e+08,None,None,None,None,-400000000.0,None,NaN,NaN,-4.170000e+09
113,2024-06-30 00:00:00+00:00,2024-08-08 00:00:00+00:00,2.01,1.6000,0.4100,25.6250,post-market,USD,5.405000e+09,6.949000e+09,...,9.720000e+08,None,None,None,None,-100000000.0,None,NaN,NaN,1.614000e+09
114,2024-09-30 00:00:00+00:00,2024-11-06 00:00:00+00:00,2.02,1.5500,0.4700,30.3226,post-market,USD,5.971000e+09,7.545000e+09,...,1.962000e+09,None,None,None,None,-300000000.0,None,NaN,NaN,1.253000e+09
115,2024-12-31 00:00:00+00:00,2025-02-04 00:00:00+00:00,1.90,1.7005,0.1995,11.7318,post-market,USD,5.988000e+09,7.569000e+09,...,9.730000e+08,None,None,None,None,-350000000.0,None,NaN,NaN,1.783000e+09
116,2025-03-31 00:00:00+00:00,2025-05-08 00:00:00+00:00,1.81,1.7726,0.0374,2.1099,post-market,None,5.706000e+09,6.667000e+09,...,1.010000e+09,None,None,None,None,-730000000.0,None,NaN,NaN,1.315000e+09
